## Introduction

Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Faiss is written in C++ with complete wrappers for Python/numpy. Some of the most useful algorithms are implemented on the GPU. It is developed primarily at Meta's [Fundamental AI Research](https://ai.facebook.com/) group.

[facebookresearch/faiss: A library for efficient similarity search and clustering of dense vectors.](https://github.com/facebookresearch/faiss)

## Installation

The simplest installation is just `pip install faiss-gpu`, or `pip install faiss-cpu`. The GPU version is faster, but has more dependencies and more likely to fail.

### Tests

We test that the installed FAISS works. The `simple_check_faiss` generates 1000 random vectors in $\mathbb R^{32}$, then check that each vector is closest to itself.

In [35]:
import numpy as np
import faiss

def simple_check_faiss():
    dimension = 32  # Dimension of the vector
    num_vectors = 1000  # Number of vectors

    # Generate some random vectors
    # np.random.seed(2)  # For reproducibility
    vectors = np.random.random((num_vectors, dimension)).astype('float32')

    # Create a Faiss index
    index = faiss.IndexFlatL2(dimension)

    # Add vectors to the index
    index.add(vectors)

    # Perform a search
    num_neighbors = 6  # Number of nearest neighbors to return
    _, indices = index.search(vectors, num_neighbors)  # Search for the first vector
    
    assert np.array_equal(indices[:, 0], np.arange(num_vectors))
    print(f"Simple check passed.")
    
simple_check_faiss()

Simple check passed.


Next, we construct a simple semantic search index over some arXiv papers. The data is stored in a `json` file `arxiv-metadata-10000.json`. It has 10000 lines, each of which is a `json` object containing metadata for a paper on arXiv. For example, the first line is (after formatting for easy reading):

```json
{
  "id": "0704.0001",
  "submitter": "Pavel Nadolsky",
  "authors": "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  "title": "Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies",
  "comments": "37 pages, 15 figures; published version",
  "journal-ref": "Phys.Rev.D76:013009,2007",
  "doi": "10.1103/PhysRevD.76.013009",
  "report-no": "ANL-HEP-PR-07-12",
  "categories": "hep-ph",
  "license": null,
  "abstract": "  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified ...",
  "versions": [ { "version": "v1", "created": "Mon, 2 Apr 2007 19:18:42 GMT" }, { "version": "v2", "created": "Tue, 24 Jul 2007 20:10:27 GMT" } ],
  "update_date": "2008-11-26",
  "authors_parsed": [ [ "Balázs", "C.", "" ], [ "Berger", "E. L.", "" ], [ "Nadolsky", "P. M.", "" ], [ "Yuan", "C. -P.", "" ] ]}
```

On my machine, DistilBERT runs at 15 entry/sec on CPU, and 100 entry/sec on GPU. It would take 5 hours to compile the entire Arxiv dataset of 1.7 million papers. The entire dataset is found in [arXiv Dataset | Kaggle](https://www.kaggle.com/datasets/Cornell-University/arxiv). It would take up 3.7 GB of space. Since this is a toy project you should not run it on the whole database.

In [93]:
import json
import numpy as np
import faiss
from transformers import DistilBertModel, DistilBertTokenizer

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DistilBERT setup
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model = model.to(device)  # Move the model to the GPU

# vectorize text batches
def vectorize_texts(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}  # Move the inputs to the GPU
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy()  # Move the result back to the CPU

# FAISS index setup
dimension = 768  # DistilBERT output dimension
index = faiss.IndexFlatL2(dimension)

# Extract and vectorize data
db_filename = 'arxiv-metadata-10000.json'
num_lines = 10000

batch_size = 16
with open(db_filename, 'r') as f:
    for _ in tqdm(range(num_lines // batch_size)):
        lines = [f.readline() for _ in range(batch_size)]
        papers = [json.loads(line) for line in lines]

        # Extract the papers' titles and abstracts
        texts = [f"{paper['title']}: {paper['abstract']}" for paper in papers]

        # Create vectors using DistilBERT
        vectors = vectorize_texts(texts)

        # Add the vectors to the FAISS index
        index.add(np.array(vectors).astype('float32'))

print(f"Added {num_lines} papers to the FAISS index.")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████████████████████████████████████████| 625/625 [02:10<00:00,  4.78it/s]

Added 10000 papers to the FAISS index.


In [112]:
def search(query, k=5):
    vector = vectorize_texts([query])
    _, indices = index.search(np.array(vector).astype('float32'), k)
    return indices[0]

def load_paper_details():
    with open(db_filename, 'r') as f:
        return [json.loads(line) for line in f]
paper_details = load_paper_details()

import latexcodec
def print_paper_details(line_numbers):
    for i in line_numbers:
        paper = paper_details[i]
        title = paper['title'].replace('\n', ' ').encode().decode('latex')
        print(f"Title: {title}")
        abstract = paper['abstract'].replace('\n', ' ').encode().decode('latex')
        print(f"Abstract: {abstract}\n")

# Test the function
query = "the statistical mechanics of freezing and boiling water"
line_numbers = search(query, k=3)
print_paper_details(line_numbers)

Title: Comparison of Dissipative Particle Dynamics and Langevin thermostats for out-of-equilibrium simulations of polymeric systems
Abstract: In this work we compare and characterize the behavior of Langevin and Dissipative Particle Dynamics (DPD) thermostats in a broad range of non-equilibrium simulations of polymeric systems. Polymer brushes in relative sliding motion, polymeric liquids in Poiseuille and Couette flows, and brush-melt interfaces are used as model systems to analyze the efficiency and limitations of different Langevin and DPD thermostat implementations. Widely used coarse-grained bead-spring models under good and poor solvent conditions are employed to assess the effects of the thermostats. We considered equilibrium, transient, and steady state examples for testing the ability of the thermostats to maintain constant temperature and to reproduce the underlying physical phenomena in non-equilibrium situations. The common practice of switching-off the Langevin thermostat 

## Instructor-XL

As we see, the search is okay. Let's try something else than DistilBERT: the `Instructor-base`. 

In short, `Instructor-base` is a language model that turns text into vectors, like DistilBERT, but unlike DistilBERT, it can be prompted. The idea is that DistilBERT is a domain-general embedder, but sometimes you need to make something domain-specific, to preserve different kinds of information. For example, an embedder for math papers should preserve distinctions between "ball" and "sphere" and "circle" and "disk", and an embedder for poetry should preserve rhyming information.

There are three models: `base`, `large`, `xl` (extra-large). We use the `base` model for experimenting as it is the fastest.



In [1]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-base')

/home/deadscholar/miniconda3/envs/faiss/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [2]:
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings.shape)

(1, 768)


In [5]:
import json
import numpy as np
import faiss
import torch
from tqdm import tqdm

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

# FAISS index setup
dimension = 768  # Instructor output dimension
index = faiss.IndexFlatL2(dimension)

# Extract and vectorize data
db_filename = 'arxiv-metadata-10000.json'
num_lines = 10000
batch_size = 4

# Preparation for encoding
instructions = ["Represent the science titles and abstracts: "] * batch_size

with open(db_filename, 'r') as f:
    for _ in tqdm(range(num_lines // batch_size)):
        lines = [f.readline() for _ in range(batch_size)]
        papers = [json.loads(line) for line in lines]

        # Extract the papers' titles and abstracts
        texts = [f"{paper['title']}: {paper['abstract']}" for paper in papers]
        
        # Prepare the inputs
        inputs = [[instr, txt] for instr, txt in zip(instructions, texts)]
        
        # Create vectors using Instructor-XL
        vectors = model.encode(
            sentences=inputs,
            batch_size=batch_size,
            show_progress_bar=False,
            convert_to_numpy=True,
            device=str(device)
        )

        # Add the vectors to the FAISS index
        index.add(np.array(vectors).astype('float32'))

print(f"Added {num_lines} papers to the FAISS index.")

device =  cuda


100%|███████████████████████████████████████████████████████████████████████████████| 2500/2500 [03:46<00:00, 11.02it/s]

Added 10000 papers to the FAISS index.
